In [4]:
def burrowWheelerTransform(t):
    ''' given string t, return Burrow-Wheeler transformed string'''
    # add $ 
    t = t + "$"
    # rotation
    rotation = []
    for i in range(len(t)):
        rotation.append(t[i:] + t[:i])
    # sort the rotation list lexicographically
    rotation.sort()
    # take the last column as the Burrow-Wheeler transformed string
    bwt = ""
    for i in range(len(rotation)):
        bwt += rotation[i][-1]
    return bwt

In [18]:
import numpy.testing as np_test
np_test.assert_equal(burrowWheelerTransform("abaaba"), "abba$aa")

In [75]:
def suffixArray(t):
    ''' given string t, return its suffix array'''
    t = t + "$"
    sa = []
    for i in range(len(t)):
        sa.append((t[i:],i))
    return sorted(sa)
def bwaBySa(t):
    ''' given string t, return Burrow-Wheeler transformed string from the suffix array'''
    bwt = ''
    for i in suffixArray(t):
        if i[1] == 0:
            bwt += "$"
        else:
            bwt += t[i[1]-1]
    return bwt

In [28]:
suffixArray("abaaba")
bwaBySa("abaaba")

'abba$aa'

In [ ]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline() # skip the name line
            #seq = fh.readline().rstrip() # read base sequence
            fh.readline() # skip placeholder line
            #qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences
seqs = readFastq("ads1_week4_reads.fq")

In [98]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [99]:
%time
a=readGenome("/root/work/pythonBase/BMI/ADS/phix.fa")
bwt = burrowWheelerTransform(a)           

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 20 µs


In [100]:
bwt

'ACTCTACCCCCCCTGCGTGGAATGAAAAATATAAGCTGCCATGAAAGAAGATAAATCCCCAAACCACTGGATTGAAAATATTTTTTTTTGCTTTGTTTAACGATTCGATGTCCCCGAAAAATACAAATATTCGATTTTGTGGCATTATCCCGACAAACCTTTTTGTCACACGTGAGTCGTAAAGACATCCCCGACACCCCTAGATACCCTAAAGAAAGAAAACGATCCGTATAACAGGCAACTGCACCCATCGAACCAGGATATGGAAGCCTGGAGAAACACACTGGGGCATTGACTACGAACATGGTTGATCGGGTCATTTCACACATACAAACCCAAAAAGTTAAGTACCATACTCGTTTGTTAGTTTCTGTAGAATATGCGAAGAACTTCATTAACAGGGAAAGACCTAAAGGTCAAGAGAATAATGTTATCTGAAAAGCGGGGGTATCTTGAGAGGTTCGTTGAGCTTAAGTATTTGACATGGGTAAGCGGAAAACTCTGGGGTTGGTGAGCAATGGACTCAACAGTGGGTAGAGCTGTCGAAGGGGAATATTACGACGTCCATGCTGTTACGATCCTCCGAGGTATTCGGCTGCTTTATTTACTGCTGTGGTGTAATGTTTGGCAGGTGTTCAGGCCGTAATGTACCGTGGTAAGCAAAGCAACGGACGCGACGCCCACTTACCAAGGCTAGAATAAAAATTTCCGAAGGGCCGGGGGGAGAAGCCAAAACGACTGAAGCAAAACAGGGGGCTCGCAAAACAAAAAAAAAAAGGGCTGCATCCCAAAGTAATGAAAAAACATTGGAAGGGACAGACGACGGGGCACGGGCAGTAGCCAAAAGATCAGCGCCTGCGCGCCGGATGCGCGAGAGTGCACACGGCCAAAACTAGCACAGGTAGGCCTGGATGGGGTGAGATACGCGGGTCGGGTTAAAGCAGTAAAGAGTAGTGGAGGTGGCGGGTGAATCACTAACAAATAACATAAACAATTATT

In [2]:
def trank(bwt):
    '''Given Burrows-Wheeler tranformed string, return
    total: a dict with chars(nucleotide) as keys and its total occurences as values
    ranks: a list of the char's rank'''
    total = dict()
    ranks = []
    for i in bwt:
        if i not in total:
            total[i] = 0
        ranks.append(total[i])
        total[i] += 1
    return total, ranks

In [10]:
def startPos(total):
    '''Given total: a dict with chars(nucleotide) as keys and its total occurences as values,
    return the postion of each char in the F column (i.e. sorted bwt) where it first occurs'''
    start_pos = dict()
    sorted_char = sorted(total)
    start_pos[sorted_char[0]] = 0
    for i in range(1,len(sorted_char)):
        start_pos[sorted_char[i]] = total[sorted_char[i-1]] +  start_pos[sorted_char[i-1]]
    return start_pos

# startPos only has the start position, we want to have start and end
def charRange(total):
    '''Given total: a dict with chars(nucleotide) as keys and its total occurences as values,
    return the postion of each char in the F column (i.e. sorted bwt) in which range it occurs (a range
    with right closed, left open)'''
    char_range = dict()
    pos = 0
    for char, count in sorted(total.items()):
        char_range[char] = (pos,pos + count)
        pos += count
    return char_range

In [5]:
bwt = burrowWheelerTransform("abaaba") #"abba$aa"
totals, ranks = trank(bwt)

In [6]:
totals

{'a': 4, 'b': 2, '$': 1}

In [9]:
startPos(totals)

{'$': 0, 'a': 1, 'b': 5}

In [43]:
charRange(totals)

{'$': (0, 1), 'a': (1, 5), 'b': (5, 7)}

In [ ]:
def findFPos(Lpos,char,ranks,start_pos):
    '''Given the pos in the L col (Lpos) and char, with ranks from trank() and start_pos
    from startPos(), return a list next_pos which is the pos in the F col that matches
    the char in the L col(like traceback)'''
    next_pos = []
    for p in Lpos:
        # ranks[p]: the rank we can jump; start_pos: the char we can jump
        next_pos.append(ranks[p]+start_pos[char])
    return next_pos

In [17]:
# def traceback(bwt):
#     '''Given Burrows-Wheeler tranformed string bwt, return
#     the origin sequence'''
#     origin_seq = ""
#     # from bwt get the first char (just sorting it)
#     #first_col = sorted(bwt)
#     # get the rank
#     total, ranks = trank(bwt)
#     start_pos = startPos(total)
#     # start from $ (first row)
#     #F = first_col[0]
#     L = bwt[0]
#     origin_seq += L
#     pos = 0
#     while L != "$":
#         # ranks[pos]: the rank we can jump; start_pos[L]: the char we can jump
#         # add them we get the new pos to traceback
#         pos = start_pos[L] + ranks[pos]
#         L = bwt[pos]
#         origin_seq += L
#     return origin_seq

In [19]:
# now rewrite traceback with charRange (replace startPos)
def traceback(bwt):
    '''Given Burrows-Wheeler tranformed string bwt, return
    the origin sequence'''
    origin_seq = ""
    # from bwt get the first char (just sorting it)
    #first_col = sorted(bwt)
    # get the rank
    total, ranks = trank(bwt)
    char_range = charRange(total)
    # start from $ (first row)
    #F = first_col[0]
    L = bwt[0]
    origin_seq += L
    pos = 0
    while L != "$":
        # ranks[pos]: the rank we can jump; char_range[L][0] : the char we can jump
        # add them we get the new pos to traceback
        pos = char_range[L][0] + ranks[pos]
        L = bwt[pos]
        origin_seq += L
    return origin_seq

In [18]:
traceback("abba$aa")

'abaaba$'

In [69]:
# def query(seq, bwt):
#     ''' Given sequence to be queried and Burrows-Wheeler tranformed string bwt,
#     return the position that seq occur in the origin string (-1 for not found)'''
#     # get the F col
#     Fcol = sorted(bwt)
#     # get the rank
#     total, ranks = trank(bwt)
#     # get the startpos
#     start_pos = startPos(total) 
    
#     # start from the last char in seq
#     first_char = seq[-1]
#     # check whether char in the ref string first
#     if first_char in total and seq[-2] in total:
#         pos = start_pos[first_char]
#         # we want to find the next pos matches
#         next_pos = []
#         while first_char == Fcol[pos]:
#             if bwt[pos] == seq[-2]:
#                 next_pos.append(pos)
#             pos += 1
#             if pos == len(bwt):
#                 break
#     else: return -1 
#     # if not found, return -1
#     if next_pos == []:
#         return -1

#     # for the remaining char, we do not need to search all start with last char,
#     # but only next_pos we get
#     for i in range(len(seq)-2, 0, -1):
#         now_char = seq[i]
#         next_char = seq[i-1]
#         # if next_char not in ref string, return -1
#         if next_char not in total:
#             return -1
#         # use new_pos to record the new next pos
#         new_pos = []
#         for p in next_pos:
#             newp = ranks[p] + start_pos[now_char]
#             if bwt[newp] == next_char:
#                 new_pos.append(newp)
#         # if not found, return -1
#         if new_pos == []:
#             return -1
#         next_pos = new_pos.copy() # update next_pos with copy
#     out_pos = []
#     for p in next_pos:
#         outp = ranks[p] + start_pos[next_char]
#         out_pos.append(outp)
#     return out_pos

In [20]:
# rewrite query with charRange (replace startPos)
def query(seq, bwt):
    ''' Given sequence to be queried and Burrows-Wheeler tranformed string bwt,
    return the position that seq occur in the origin string (-1 for not found)'''
    # get the F col
    Fcol = sorted(bwt)
    # get the rank
    total, ranks = trank(bwt)
    # get the start and end pos of each char
    char_range = charRange(total) 
    
    # start from the last char in seq
    first_char = seq[-1]
    # check whether char in the ref string first
    if first_char in total and seq[-2] in total:
        pos = char_range[first_char][0]
        # we want to find the next pos matches
        next_pos = []
        while first_char == Fcol[pos]:
            if bwt[pos] == seq[-2]:
                next_pos.append(pos)
            pos += 1
            if pos == len(bwt):
                break
    else: return -1 
    # if not found, return -1
    if next_pos == []:
        return -1

    # for the remaining char, we do not need to search all start with last char,
    # but only next_pos we get
    for i in range(len(seq)-2, 0, -1):
        now_char = seq[i]
        next_char = seq[i-1]
        # if next_char not in ref string, return -1
        if next_char not in total:
            return -1
        # use new_pos to record the new next pos
        new_pos = []
        for p in next_pos:
            newp = ranks[p] + char_range[now_char][0]
            if bwt[newp] == next_char:
                new_pos.append(newp)
        # if not found, return -1
        if new_pos == []:
            return -1
        next_pos = new_pos.copy() # update next_pos with copy
    out_pos = []
    for p in next_pos:
        outp = ranks[p] + char_range[next_char][0]
        out_pos.append(outp)
    return out_pos

In [24]:
query("abaaba",burrowWheelerTransform("abaaba"))

[4]

In [67]:
burrowWheelerTransform("abaaba")

'abba$aa'

In [28]:
# Scanning for preceding character is slow
# Solution: pre-calculate the number of char in L col in each row --- tally
def generateTally(bwt):
    ''' Burrow-Wheeler transformed string, return a dict called all_char_tally,
    key: char, value: its count from start to end (a list)'''
    tally = dict()
    all_char_tally = dict()
    for i in bwt:
        if i not in tally:
            tally[i] = 0
            all_char_tally[i] = []
    for rowindex, char in enumerate(bwt):
        tally[char] += 1
        for c in tally.keys():
            all_char_tally[c].append(tally[c])
    return all_char_tally

In [29]:
generateTally(burrowWheelerTransform("abaaba"))

{'a': [1, 1, 1, 2, 2, 3, 4],
 'b': [0, 1, 2, 2, 2, 2, 2],
 '$': [0, 0, 0, 0, 1, 1, 1]}

In [44]:
def findNextWithTally(current_range, next_char,tally,char_range):
    '''Given the current range (left closed, right open) in the F col and the next char searching for, with the help of tally and char_range,
    return the rank of the next char satified (-1 for not found)'''
    start = current_range[0] - 1 # need to see on before so that we do not leave out the first char
    end = current_range[1] - 1 # right open
    if next_char not in tally:
        return -1
    num_char_found = tally[next_char][end] - tally[next_char][start] # how many next char found
    if num_char_found > 0:
        rank_start, rank_end = tally[next_char][start], tally[next_char][end]-1
        return [char_range[next_char][0]+rank_start, char_range[next_char][0]+rank_end+1] # range is left open, right closed
    else:
        return -1

In [64]:
# rewrite query with tally
def queryWithTally(seq, bwt):
    ''' Given sequence to be queried and Burrows-Wheeler tranformed string bwt,
    return the position that seq occur in the origin string (-1 for not found)'''
    # get the F col
    Fcol = sorted(bwt)
    # get the rank
    total, ranks = trank(bwt)
    # get the start and end pos of each char
    char_range = charRange(total) 
    # get the tally for all chars
    all_tally = generateTally(bwt)
    
    # start from the last char in seq
    first_char = seq[-1]
    # check whether char in the ref string first
    if first_char not in total:
        return -1
    # the range of the first char can be found in char_range
    cc_range = [char_range[first_char][0], char_range[first_char][1]]
    # length of seq is 1 (uncommon case)
    if len(seq) == 1:
        return cc_range
    # for the remaining char in seq
    for i in range(len(seq)-2,-1,-1):
        cc_range = findNextWithTally(cc_range, seq[i], all_tally, char_range)
        print(seq[i],cc_range)
        if cc_range == -1:
            return -1 
    return cc_range

In [74]:
queryWithTally("c",burrowWheelerTransform("abcabc"))

[5, 7]

In [106]:
# we need suffix array to where the aligned seq is in the origin string
def querySA(seq, t):
    ''' Given sequence to be queried and origin string t, performing the BWT,
    return the position that seq occur in the origin string (-1 for not found)'''
    bwt = burrowWheelerTransform(t)
    sa = [i[1] for i in suffixArray(t)]
    # get the F col
    Fcol = sorted(bwt)
    # get the rank
    total, ranks = trank(bwt)
    # get the start and end pos of each char
    char_range = charRange(total) 
    # get the tally for all chars
    all_tally = generateTally(bwt)
    
    # start from the last char in seq
    first_char = seq[-1]
    # check whether char in the ref string first
    if first_char not in total:
        return -1
    # the range of the first char can be found in char_range
    cc_range = [char_range[first_char][0], char_range[first_char][1]]
    # length of seq is 1 (uncommon case)
    if len(seq) == 1:
        return cc_range
    # for the remaining char in seq
    for i in range(len(seq)-2,-1,-1):
        cc_range = findNextWithTally(cc_range, seq[i], all_tally, char_range)
        #print(seq[i],cc_range)
        if cc_range == -1:
            return -1 
    return sa[cc_range[0]:cc_range[1]]

In [110]:
querySA("ba","abacaba")

[5, 1]

In [ ]:
%time
a=readGenome("data/chr1.GRCh38.excerpt.fasta")
bwt = burrowWheelerTransform(a) 

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 15 µs


In [117]:
! wget https://sra-download.ncbi.nlm.nih.gov/traces/sra20/SRR/015102/SRR15465183

--2021-12-11 10:13:15--  https://sra-download.ncbi.nlm.nih.gov/traces/sra20/SRR/015102/SRR15465183
Connecting to 10.71.115.244:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 39423721 (38M) [application/octet-stream]
Saving to: ‘SRR15465183’

SRR15465183         100%[===================>]  37.60M  1.92MB/s    in 28s     

2021-12-11 10:13:49 (1.34 MB/s) - ‘SRR15465183’ saved [39423721/39423721]



In [116]:
%time
querySA("ATCG",a)[1:10]

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 20.3 µs


KeyboardInterrupt: 